<a href="https://colab.research.google.com/github/rupaai/caffeine_coders_emotion_recognition/blob/master/downloadTweets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
import seaborn as sns
import re
import collections
from wordcloud import WordCloud

In [2]:
### read data from your Google Drive
from google.colab import drive
drive.mount('/gdrive')

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [4]:
from tweepy.streaming import StreamListener
from tweepy import OAuthHandler
from tweepy import Stream
import re

# Enter Twitter API Keys
access_token = "ACCESS_TOKEN"
access_token_secret = "ACCESS_TOKEN_SECRET"
consumer_key = "CONSUMER_KEY"
consumer_secret = "CONSUMER_KEY_SECRET"

# Create tracklist with the words that will be searched for
tracklist = ['#udacity', "#facebookai","pytorch","#facebook"]
# Initialize Global variable
tweet_count = 0
# Input number of tweets to be downloaded
n_tweets = 100

tweets = []
# Create the class that will handle the tweet stream
class StdOutListener(StreamListener):
      
    def on_data(self, data):
        global tweet_count
        global n_tweets
        global stream
        #if tweet_count == 0:
        #    f = open("tweets.txt","w+")
            
        if tweet_count < n_tweets:
            print(data)
            #f.write(data)
            tweets.append(data)
            tweet_count += 1
            return True
        else:
            stream.disconnect()
            #f.close()

    def on_error(self, status):
        print(status)



# Handles Twitter authetification and the connection to Twitter Streaming API
l = StdOutListener()
auth = OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
stream = Stream(auth, l)
stream.filter(track=tracklist)

if(len(tweets) > 0):
  print("writing data to a file")
  f = open("/gdrive/My Drive/Colab Notebooks/tweets.txt","w+")
  for tweet in tweets:
      f.write(tweet)
  
  f.close()
  print("data successfully written")

{"created_at":"Tue Aug 20 19:49:05 +0000 2019","id":1163900781008949248,"id_str":"1163900781008949248","text":"RT @jepence: An audit of #Facebook found that there is \"still SIGNIFICANT WORK to be done to satisfy.. concerns\" of Conservative bias\u2013\u2013mean\u2026","source":"\u003ca href=\"http:\/\/twitter.com\/download\/iphone\" rel=\"nofollow\"\u003eTwitter for iPhone\u003c\/a\u003e","truncated":false,"in_reply_to_status_id":null,"in_reply_to_status_id_str":null,"in_reply_to_user_id":null,"in_reply_to_user_id_str":null,"in_reply_to_screen_name":null,"user":{"id":1073687093988216832,"id_str":"1073687093988216832","name":"DOITQBABY","screen_name":"doitqbaby","location":"United States","url":null,"description":"THE MORE YOU RESEARCH THE CRAZIER YOU SOUND TO IGNORANT PEOPLE... #WWG1WGA","translator_type":"none","protected":false,"verified":false,"followers_count":2616,"friends_count":3131,"listed_count":1,"favourites_count":1471,"statuses_count":1842,"created_at":"Fri Dec 14 21:12:06

In [5]:
import json
import pandas as pd
tweets_data_path = "/gdrive/My Drive/Colab Notebooks/tweets.txt"  
tweets_data = []  
tweets_file = open(tweets_data_path, "r")  
for line in tweets_file:  
    try:  
        tweet = json.loads(line)  
        tweets_data.append(tweet)  
    except:  
        continue
tweets = pd.DataFrame()
tweets['text'] = list(map(lambda tweet: tweet['text'], tweets_data))
tweets['Username'] = list(map(lambda tweet: tweet['user']['screen_name'], tweets_data))
tweets['Timestamp'] = list(map(lambda tweet: tweet['created_at'], tweets_data))
tweets.head()

,text,Username,Timestamp
0,RT @jepence: An audit of #Facebook found that ...,doitqbaby,Tue Aug 20 19:49:05 +0000 2019
1,"RT @Brett_Shavers: Only at #Facebook does ""cle...",rjacksix,Tue Aug 20 19:49:09 +0000 2019
2,RT @jepence: An audit of #Facebook found that ...,kenneth62678789,Tue Aug 20 19:49:19 +0000 2019
3,RT @EmilWallner: The AI autodidact degree.\n\n...,twtAtul,Tue Aug 20 19:49:20 +0000 2019
4,🎵💫 2Day‼️🙌 🎶🎼 From Jazz To House: https://t.co...,NINI_ETERNALSOL,Tue Aug 20 19:49:23 +0000 2019


In [0]:
#Reading the raw data collected from the Twitter Streaming API using #Tweepy. 
tweets_data = []
tweets_data_path = '/gdrive/My Drive/Colab Notebooks/tweets.txt'
tweets_file = open(tweets_data_path, "r")
for line in tweets_file:
    try:
        tweet = json.loads(line)
        tweets_data.append(tweet)
    except:
        continue

In [0]:
#Error codes from the Twitter API can be inside the .txt document, #take them off
tweets_data = [x for x in tweets_data if not isinstance(x, int)]

In [8]:
print("The total number of Tweets is:",len(tweets_data))

The total number of Tweets is: 100


In [0]:
#Create a function to see if the tweet is a retweet
def is_RT(tweet):
    if 'retweeted_status' not in tweet:
        return False      
    else:
        return True

In [0]:
#Create a function to see if the tweet is a reply to a tweet of #another user, if so return said user. 
def is_Reply_to(tweet):
    if 'in_reply_to_screen_name' not in tweet:
        return False      
    else:
        return tweet['in_reply_to_screen_name']

In [0]:
#Create function for taking the most used Tweet sources off the #source column 
def reckondevice(tweet):
    if 'iPhone' in tweet['source'] or ('iOS' in tweet['source']):
        return 'iPhone'
    elif 'Android' in tweet['source']:
        return 'Android'
    elif 'Mobile' in tweet['source'] or ('App' in tweet['source']):
        return 'Mobile device'
    elif 'Mac' in tweet['source']:
        return 'Mac'
    elif 'Windows' in tweet['source']:
        return 'Windows'
    elif 'Bot' in tweet['source']:
        return 'Bot'
    elif 'Web' in tweet['source']:
        return 'Web'
    elif 'Instagram' in tweet['source']:
        return 'Instagram'
    elif 'Blackberry' in tweet['source']:
        return 'Blackberry'
    elif 'iPad' in tweet['source']:
        return 'iPad'
    elif 'Foursquare' in tweet['source']:
        return 'Foursquare'
    else:
        return '-'

In [0]:
#Convert the Tweet JSON data to a pandas Dataframe, and take the #desired fields from the JSON. More could be added if needed.
tweets = pd.DataFrame()
tweets['text'] = list(map(lambda tweet: tweet['text'] if 'extended_tweet' not in tweet else tweet['extended_tweet']['full_text'], tweets_data))
tweets['Username'] = list(map(lambda tweet: tweet['user']['screen_name'], tweets_data))
tweets['Timestamp'] = list(map(lambda tweet: tweet['created_at'], tweets_data))
tweets['lenght'] = list(map(lambda tweet: len(tweet['text']) if'extended_tweet' not in tweet else len(tweet['extended_tweet']['full_text']) , tweets_data))
tweets['location'] = list(map(lambda tweet: tweet['user']['location'], tweets_data))
tweets['device'] = list(map(reckondevice, tweets_data))
tweets['RT'] = list(map(is_RT, tweets_data))
tweets['Reply'] = list(map(is_Reply_to, tweets_data))

In [13]:
tweets.head()

,text,Username,Timestamp,lenght,location,device,RT,Reply
0,RT @jepence: An audit of #Facebook found that ...,doitqbaby,Tue Aug 20 19:49:05 +0000 2019,140,United States,iPhone,True,None
1,"RT @Brett_Shavers: Only at #Facebook does ""cle...",rjacksix,Tue Aug 20 19:49:09 +0000 2019,131,"Helena, MT",Android,True,None
2,RT @jepence: An audit of #Facebook found that ...,kenneth62678789,Tue Aug 20 19:49:19 +0000 2019,140,None,iPhone,True,None
3,RT @EmilWallner: The AI autodidact degree.\n\n...,twtAtul,Tue Aug 20 19:49:20 +0000 2019,139,"Richmond, VA",Android,True,None
4,🎵💫 2Day‼️🙌 🎶🎼 From Jazz To House: https://t.co...,NINI_ETERNALSOL,Tue Aug 20 19:49:23 +0000 2019,175,NYC & Global,Instagram,False,None


In [14]:
#See the percentage of tweets from the initial set that are #retweets:
RT_tweets = tweets[tweets['RT'] == True]
print(f"The percentage of retweets is {round(len(RT_tweets)/len(tweets)*100)}% of all the tweets")

The percentage of retweets is 63% of all the tweets


In [15]:
RT_tweets.head()

,text,Username,Timestamp,lenght,location,device,RT,Reply
0,RT @jepence: An audit of #Facebook found that ...,doitqbaby,Tue Aug 20 19:49:05 +0000 2019,140,United States,iPhone,True,None
1,"RT @Brett_Shavers: Only at #Facebook does ""cle...",rjacksix,Tue Aug 20 19:49:09 +0000 2019,131,"Helena, MT",Android,True,None
2,RT @jepence: An audit of #Facebook found that ...,kenneth62678789,Tue Aug 20 19:49:19 +0000 2019,140,None,iPhone,True,None
3,RT @EmilWallner: The AI autodidact degree.\n\n...,twtAtul,Tue Aug 20 19:49:20 +0000 2019,139,"Richmond, VA",Android,True,None
5,RT @jepence: An audit of #Facebook found that ...,mpitts27,Tue Aug 20 19:49:26 +0000 2019,140,None,iPad,True,None


In [16]:
#See the percentage of tweets from the initial set that are replies #to tweets of another user:
Reply_tweets = tweets[tweets['Reply'].apply(type) == str]
print(f"The percentage of retweets is {round(len(Reply_tweets)/len(tweets)*100)}% of all the tweets")

The percentage of retweets is 4% of all the tweets
